In [3]:
pip install pandas

  Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os 
import pandas as pd
import numpy as np
import json
from multiprocessing import Pool,cpu_count

In [2]:

with open(r'E:\MediNexus\Datasets\Symptoms2Dis\release_evidences.json', 'r') as f:
    evidences = json.load(f)

with open(r'E:\MediNexus\Datasets\Symptoms2Dis\release_conditions.json', 'r') as f:
    conditions = json.load(f)

# Create dictionaries for translations
condition_translations = {cond['condition_name']: cond['cond-name-eng'] for cond in conditions.values()}



In [3]:
disease=[]
question=[]
symptoms=[]
Datatype=[]
DefaultVal=[]
Is_antecedent=[]
disease_symptom_part={} # dictionary that stores the conversion of french to engllish body parts
for i in evidences.keys():
    disease.append(i)
    question.append(evidences[i]['question_en'])
    Datatype.append(evidences[i]['data_type'])
    dis_val=evidences[i]['value_meaning']
    Is_antecedent.append(evidences[i]['is_antecedent'])
    DefaultVal.append(evidences[i]['default_value'])
    print(dis_val)
    perdiseasesymptom=[]
    for j in dis_val.keys():
        eng_symptom=dis_val[j]['en']
        perdiseasesymptom.append(eng_symptom)
        if j not in disease_symptom_part.keys():
            disease_symptom_part[j]=eng_symptom
    symptoms.append(perdiseasesymptom)       

{}
{'nulle_part': {'fr': 'nulle part', 'en': 'nowhere'}, 'aile_iliaque_D_': {'fr': 'aile iliaque(D)', 'en': 'iliac wing(R)'}, 'aile_iliaque_G_': {'fr': 'aile iliaque(G)', 'en': 'iliac wing(L)'}, 'aine_D_': {'fr': 'aine(D)', 'en': 'groin(R)'}, 'aine_G_': {'fr': 'aine(G)', 'en': 'groin(L)'}, 'aisselle_G_': {'fr': 'aisselle(G)', 'en': 'axilla(L)'}, 'aisselle_D_': {'fr': 'aisselle(D)', 'en': 'axilla(R)'}, 'amygdale_D_': {'fr': 'amygdale(D)', 'en': 'tonsil(R)'}, 'amygdale_G_': {'fr': 'amygdale(G)', 'en': 'tonsil(L)'}, 'anus': {'fr': 'anus', 'en': 'anus'}, 'arrière_du_cou': {'fr': 'arrière du cou', 'en': 'back of the neck'}, 'arrière_de_tête': {'fr': 'arrière de tête', 'en': 'back of head'}, 'biceps_D_': {'fr': 'biceps(D)', 'en': 'biceps(R)'}, 'biceps_G_': {'fr': 'biceps(G)', 'en': 'biceps(L)'}, 'bouche': {'fr': 'bouche', 'en': 'mouth'}, 'cartilage_thyroidien': {'fr': 'cartilage thyroidien', 'en': 'thyroid cartilage'}, 'cheville_D_': {'fr': 'cheville(D)', 'en': 'ankle(R)'}, 'cheville_G_': {'

In [4]:
print(len(disease))
print(len(question))
print(len(symptoms))
print(len(Datatype))
print(len(DefaultVal))
print(len(Is_antecedent))

223
223
223
223
223
223


In [5]:
evidences_dict={
    'DiseaseName':disease,
    'question_en':question,
     'symptom':symptoms,
    'datatype':Datatype,
      'default_value':DefaultVal,
    'AntecedentOrNot':Is_antecedent
}

In [6]:
evidences_df=pd.DataFrame(evidences_dict)

In [7]:
list(evidences_df['DiseaseName'])
list(evidences_df['question_en'])



['Do you have a fever (either felt or measured with a thermometer)?',
 'Do you feel pain somewhere?',
 'Do you have pain somewhere, related to your reason for consulting?',
 'Does the pain radiate to another location?',
 'Characterize your pain:',
 'How fast did the pain appear?',
 'How intense is the pain?',
 'How precisely is the pain located?',
 'Did you lose consciousness?',
 'Where is the affected region located?',
 'Do you have any lesions, redness or problems on your skin that you believe are related to the condition you are consulting for?',
 'What color is the rash?',
 'How intense is the pain caused by the rash?',
 'Is the rash swollen?',
 'How severe is the itching?',
 'Is the lesion (or are the lesions) larger than 1cm?',
 'Do your lesions peel off?',
 'Is your skin much paler than usual?',
 'Do you feel your heart is beating fast (racing), irregularly (missing a beat) or do you feel palpitations?',
 'Have you recently thrown up blood or something resembling coffee beans?',

In [8]:
filt_multi=evidences_df['datatype']=='M'
filt_cat=evidences_df['datatype']=='C'
multi_symp_df=evidences_df.loc[filt_multi,:]
cat_symp_df=evidences_df.loc[filt_cat,:]
  

In [9]:
multi_symp_df

,DiseaseName,question_en,symptom,datatype,default_value,AntecedentOrNot
1,douleurxx_endroitducorps,Do you feel pain somewhere?,"[nowhere, iliac wing(R), iliac wing(L), groin(...",M,nulle_part,False
3,douleurxx_irrad,Does the pain radiate to another location?,"[nowhere, iliac wing(R), iliac wing(L), groin(...",M,nulle_part,False
4,douleurxx_carac,Characterize your pain:,"[NA, heartbreaking, haunting, tedious, sensiti...",M,NA,False
9,lesions_peau_endroitducorps,Where is the affected region located?,"[nowhere, iliac wing(R), iliac wing(L), groin(...",M,nulle_part,False
76,oedeme_endroitducorps,Where is the swelling located?,"[nowhere, iliac wing(R), iliac wing(L), groin(...",M,nulle_part,False


In [10]:
cat_symp_df

,DiseaseName,question_en,symptom,datatype,default_value,AntecedentOrNot
5,douleurxx_soudain,How fast did the pain appear?,[],C,0,False
6,douleurxx_intens,How intense is the pain?,[],C,0,False
7,douleurxx_precis,How precisely is the pain located?,[],C,0,False
11,lesions_peau_couleur,What color is the rash?,"[NA, dark, yellow, pale, pink, red]",C,NA,False
12,lesions_peau_intens,How intense is the pain caused by the rash?,[],C,0,False
13,lesions_peau_elevee,Is the rash swollen?,[],C,0,False
14,lesions_peau_prurit,How severe is the itching?,[],C,0,False
15,lesions_peau_plusqu1cm,Is the lesion (or are the lesions) larger than...,"[N, Y]",C,N,False
16,lesions_peau_desquame,Do your lesions peel off?,"[N, Y]",C,N,False
138,trav1,Have you traveled out of the country in the la...,"[N, North Africa, West Africa, South Africa, C...",C,N,True


In [11]:
len(evidences_df)

223

In [15]:
filt=evidences_df['datatype']=='C'
evidences_df.loc[filt,'DiseaseName']

5           douleurxx_soudain
6            douleurxx_intens
7            douleurxx_precis
11       lesions_peau_couleur
12        lesions_peau_intens
13        lesions_peau_elevee
14        lesions_peau_prurit
15     lesions_peau_plusqu1cm
16      lesions_peau_desquame
138                     trav1
Name: DiseaseName, dtype: object

In [36]:
real_Df=pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv')

In [37]:
real_Df

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,18,"[['Bronchite', 0.19171203430383882], ['Pneumon...",M,IVRS ou virémie,"['crowd', 'diaph', 'douleurxx', 'douleurxx_car...",fievre
1,21,"[['VIH (Primo-infection)', 0.5189500564407601]...",M,VIH (Primo-infection),"['adp_dlr', 'atcd_its', 'diaph', 'diarrhee', '...",diaph
2,19,"[['Bronchite', 0.11278064619119596], ['Pneumon...",F,Pneumonie,"['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",expecto
3,34,"[['IVRS ou virémie', 0.23859396799565236], ['C...",F,IVRS ou virémie,"['crowd', 'douleurxx', 'douleurxx_carac_@_une_...",douleurxx
4,36,"[['IVRS ou virémie', 0.23677812769175735], ['P...",M,IVRS ou virémie,"['dayc', 'diaph', 'douleurxx', 'douleurxx_cara...",toux
...,...,...,...,...,...,...
1025597,18,"[['Épiglottite', 0.28156957795466475], ['VIH (...",M,Épiglottite,"['bw_bending', 'douleurxx', 'douleurxx_carac_@...",fievre
1025598,28,"[['Épiglottite', 0.3703962237298842], ['Laryng...",F,Épiglottite,"['douleurxx', 'douleurxx_carac_@_vive', 'doule...",fievre
1025599,0,"[['Épiglottite', 0.13193905052537108], ['Laryn...",F,Épiglottite,"['bw_bending', 'douleurxx', 'douleurxx_carac_@...",stridor
1025600,26,"[['Épiglottite', 0.3028258988138983], ['Laryng...",F,Épiglottite,"['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",stridor


In [22]:
evidences_df['datatype']=='C'

0      False
1      False
2      False
3      False
4      False
       ...  
218    False
219    False
220    False
221    False
222    False
Name: datatype, Length: 223, dtype: bool

In [50]:
text='douleurxx_endroitducorps_@_thorax_postérieur_D_'
for i in text.split('_@_'):
    print(i)
    

douleurxx_endroitducorps
thorax_postérieur_D_


In [64]:
if 'a'.isdigit():
    print(2)

In [11]:
real_df=pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv')

In [13]:
real_df

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,18,"[['Bronchite', 0.19171203430383882], ['Pneumon...",M,IVRS ou virémie,"['crowd', 'diaph', 'douleurxx', 'douleurxx_car...",fievre
1,21,"[['VIH (Primo-infection)', 0.5189500564407601]...",M,VIH (Primo-infection),"['adp_dlr', 'atcd_its', 'diaph', 'diarrhee', '...",diaph
2,19,"[['Bronchite', 0.11278064619119596], ['Pneumon...",F,Pneumonie,"['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",expecto
3,34,"[['IVRS ou virémie', 0.23859396799565236], ['C...",F,IVRS ou virémie,"['crowd', 'douleurxx', 'douleurxx_carac_@_une_...",douleurxx
4,36,"[['IVRS ou virémie', 0.23677812769175735], ['P...",M,IVRS ou virémie,"['dayc', 'diaph', 'douleurxx', 'douleurxx_cara...",toux
...,...,...,...,...,...,...
1025597,18,"[['Épiglottite', 0.28156957795466475], ['VIH (...",M,Épiglottite,"['bw_bending', 'douleurxx', 'douleurxx_carac_@...",fievre
1025598,28,"[['Épiglottite', 0.3703962237298842], ['Laryng...",F,Épiglottite,"['douleurxx', 'douleurxx_carac_@_vive', 'doule...",fievre
1025599,0,"[['Épiglottite', 0.13193905052537108], ['Laryn...",F,Épiglottite,"['bw_bending', 'douleurxx', 'douleurxx_carac_@...",stridor
1025600,26,"[['Épiglottite', 0.3028258988138983], ['Laryng...",F,Épiglottite,"['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",stridor


In [25]:
evidences_df

,DiseaseName,question_en,symptom,datatype,default_value,AntecedentOrNot
0,fievre,Do you have a fever (either felt or measured w...,[],B,0,False
1,douleurxx_endroitducorps,Do you feel pain somewhere?,"[nowhere, iliac wing(R), iliac wing(L), groin(...",M,nulle_part,False
2,douleurxx,"Do you have pain somewhere, related to your re...",[],B,0,False
3,douleurxx_irrad,Does the pain radiate to another location?,"[nowhere, iliac wing(R), iliac wing(L), groin(...",M,nulle_part,False
4,douleurxx_carac,Characterize your pain:,"[NA, heartbreaking, haunting, tedious, sensiti...",M,NA,False
...,...,...,...,...,...,...
218,suburb,Do you live in the suburbs?,[],B,0,True
219,rural,Do you live in a rural area?,[],B,0,True
220,z80.1,Do you have family members who have had lung c...,[],B,0,True
221,z80.0,Are there members of your family who have been...,[],B,0,True


In [16]:
print(evidences_df.columns)


Index(['DiseaseName', 'question_en', 'symptom', 'datatype', 'default_value',
       'AntecedentOrNot'],
      dtype='object')


In [17]:
filt1 = evidences_df['DiseaseName'] == 'diaph'
print((evidences_df.loc[filt1,'datatype'] =='B'))

52    True
Name: datatype, dtype: bool


In [15]:
import pandas as pd

def evidence_dispersor(row):
    for i in row:
        filt1 = evidences_df['DiseaseName'] == i
        if filt1.any():  # Check if filt1 has any True values
            if evidences_df.loc[filt1, 'datatype'].iloc[0] == 'B':
                row[i] = 1
            else:
                parts = i.split('_@_')
                if parts[1].isdigit():
                    print(part[1])
                    row[parts[0]] = int(parts[1])
                else:
                    row[i] = 1
    return row

modified_chunks = []
for chunk in pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv', chunksize=80000):
    modified_chunk = chunk.apply(lambda row: evidence_dispersor(row), axis=1)
    print(type(modified_chunk))
    modified_chunks.append(modified_chunk)

# Concatenate the modified chunks into a single DataFrame
modified_df = pd.concat(modified_chunks, ignore_index=True)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


KeyboardInterrupt: 

In [18]:
modified_df.to_csv(r'E:\MediNexus\Datasets\modified.csv')

In [ ]:

def evidence_dispersor(row):
    for i in row:
        print(i)
        filt1=evidences_df['DiseaseName']==i #change disese name attribute to symptom name and symptom to characterstics of symptom
        if(evidences_df.loc[filt1,'datatype'].iloc[0]=='B'):
            row[i]=1
        else:
            parts=i.split('_@_')
            if parts[1].isdigit():
                row[parts[0]]=int(parts[1])
            else:
                row[i]=1
        return row
def process_chunk(chunk):
    return chunk.apply(evidence_dispersor, axis=1)

if __name__ == "__main__":
    chunksize = 80000  # Adjust chunk size as needed
    num_processes = cpu_count()  # Number of CPU cores

    with Pool(num_processes) as pool:
        chunks = pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv', chunksize=chunksize)
        results = pool.map(process_chunk, chunks)

    # Concatenate the results into a single DataFrame or process them further
    processed_data = pd.concat(results)
            

In [82]:
for chunk in pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv',chunksize=1000):
     chunk=chunk.apply(evidence_dispersor(row))

0      ['crowd', 'diaph', 'douleurxx', 'douleurxx_car...
1      ['adp_dlr', 'atcd_its', 'diaph', 'diarrhee', '...
2      ['douleurxx', 'douleurxx_carac_@_un_coup_de_co...
3      ['crowd', 'douleurxx', 'douleurxx_carac_@_une_...
4      ['dayc', 'diaph', 'douleurxx', 'douleurxx_cara...
                             ...                        
995    ['adp_dlr', 'douleurxx', 'douleurxx_carac_@_un...
996    ['bw_bending', 'douleurxx', 'douleurxx_carac_@...
997    ['douleurxx', 'douleurxx_carac_@_une_lourdeur_...
998    ['anxiete_s', 'douleurxx_carac_@_NA', 'douleur...
999    ['douleurxx', 'douleurxx_carac_@_sensible', 'd...
Name: EVIDENCES, Length: 1000, dtype: object
1000    ['crowd', 'douleurxx', 'douleurxx_carac_@_péni...
1001    ['angor_repos', 'ap_pneumothorax', 'douleurxx'...
1002    ['allergie_sev', 'diarrhee', 'douleurxx', 'dou...
1003    ['contact', 'crowd', 'dayc', 'diaph', 'douleur...
1004    ['J05.0', 'dyspn', 'rhino_clair', 'sialorhee',...
                              ...     

In [77]:
test='daaf_@_5'
parts=test.split('_@_')

In [3]:
modified_df=pd.read_csv(r'E:\MediNexus\Datasets\modified.csv')

In [18]:
len(modified_df.columns)

103

In [44]:
modified_df.iloc[:10]

,Unnamed: 0,AGE,DIFFERENTIAL_DIAGNOSIS,EVIDENCES,INITIAL_EVIDENCE,PATHOLOGY,SEX,adp_dlr,angor_accelere,angor_repos,...,vo_violent,volume_parole,wheez,ww_bouffe,ww_bouger,ww_dd,ww_effort,ww_nuit,ww_respi,ww_valsalva
0,0,18,"[['Bronchite', 0.19171203430383882], ['Pneumon...","['crowd', 'diaph', 'douleurxx', 'douleurxx_car...",fievre,IVRS ou virémie,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,21,"[['VIH (Primo-infection)', 0.5189500564407601]...","['adp_dlr', 'atcd_its', 'diaph', 'diarrhee', '...",diaph,VIH (Primo-infection),M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,19,"[['Bronchite', 0.11278064619119596], ['Pneumon...","['douleurxx', 'douleurxx_carac_@_un_coup_de_co...",expecto,Pneumonie,F,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,34,"[['IVRS ou virémie', 0.23859396799565236], ['C...","['crowd', 'douleurxx', 'douleurxx_carac_@_une_...",douleurxx,IVRS ou virémie,F,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,36,"[['IVRS ou virémie', 0.23677812769175735], ['P...","['dayc', 'diaph', 'douleurxx', 'douleurxx_cara...",toux,IVRS ou virémie,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,37,"[['Rhinosinusite aigue', 0.2575738638873495], ...","['douleurxx', 'douleurxx_carac_@_une_brûlure_o...",douleurxx,Rhinosinusite chronique,F,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,43,"[['Bronchite', 0.24726070382402293], ['Laryngi...","['contact', 'dayc', 'douleurxx', 'douleurxx_ca...",douleurxx,Pharyngite virale,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,39,"[['Anémie', 0.35566843677250953], ['Anaphylaxi...","['Mauv_aliment', 'atcd_anem', 'douleurxx', 'do...",fatig_ext,Anémie,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,30,"[['Bronchite', 0.33250589804545644], ['Rhinosi...","['douleurxx', 'douleurxx_carac_@_vive', 'doule...",douleurxx,Rhinosinusite chronique,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,49,"[['TSVP', 0.14665916736864038], ['Anémie', 0.1...","['ap_hypert4', 'ap_valve', 'bode', 'dyspn', 'e...",dyspn,Fibrillation auriculaire/Flutter auriculaire,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:

def evidence_dispersor(row, all_evidence_names):
    for i in row['EVIDENCES']:
        filt1 = evidences_df['DiseaseName'] == i
        if filt1.any():
            if evidences_df.loc[filt1, 'datatype'].iloc[0] == 'B':
                row[i] = 1
            else:
                parts = i.split('_@_')
                if parts[1].isdigit():
                    print(parts[1])
                    row[parts[0]] = int(parts[1])

                else:
                    row[i] = 1
        else:
            row[i] = 0  # Set to 0 if evidence is not present

    # Set all other evidences to 0
    for ev_name in all_evidence_names:
        if ev_name not in row.index:
            row[ev_name] = 0

    return row

In [13]:
# Create a DataFrame with all 223 evidence columns
all_evidence_names = list(evidences_df['DiseaseName'].unique())
processed_data = pd.DataFrame(columns=['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'INITIAL_EVIDENCE'] + all_evidence_names)



In [38]:
fiit=processed_data['z80.1']==1'
processed_data.loc[fiit,'z80.1']

Series([], Name: z80.1, dtype: int64)

In [14]:
processed_data

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,INITIAL_EVIDENCE,fievre,douleurxx_endroitducorps,douleurxx,douleurxx_irrad,douleurxx_carac,...,tmine,tconst,j34.2,j33,momasthma,suburb,rural,z80.1,z80.0,J81


In [15]:
all_evidence_names

['fievre',
 'douleurxx_endroitducorps',
 'douleurxx',
 'douleurxx_irrad',
 'douleurxx_carac',
 'douleurxx_soudain',
 'douleurxx_intens',
 'douleurxx_precis',
 'pdc',
 'lesions_peau_endroitducorps',
 'lesions_peau',
 'lesions_peau_couleur',
 'lesions_peau_intens',
 'lesions_peau_elevee',
 'lesions_peau_prurit',
 'lesions_peau_plusqu1cm',
 'lesions_peau_desquame',
 'pale',
 'palpit',
 'vo_sg',
 'melena',
 'diarrhee',
 'etouff',
 'fatig_mod',
 'irritable',
 'faible',
 'nausee',
 'friss',
 'ww_respi',
 'perte_appet',
 'rectorragie',
 'perte_poids',
 'pyrosis',
 'bw_bending',
 'ww_effort',
 'footnumb',
 'dyspn',
 'pertes_vag',
 'ballon_abdo',
 'larmes',
 'rhino_clair',
 'fatig_ext',
 'convulsion',
 'paralysie_visage',
 'msk_dlr',
 'ww_bouger',
 'toux',
 'ww_dd',
 'ww_bouffe',
 'dysp_effort',
 'gain_poids',
 'anxiete_s',
 'diaph',
 'gorge_dlr',
 'adp_dlr',
 'etourdissement',
 'hta',
 'dysphagie',
 'erytheme_occ',
 'trismus',
 'dysarthrie',
 'laryngospasme',
 'sialorhee',
 'confusion',
 'volu

In [21]:
def process_chunk(chunk):
    chunk = chunk.join(processed_data[all_evidence_names].iloc[0:0])  # Add empty columns for evidences
    chunk = chunk.apply(evidence_dispersor, args=(all_evidence_names,), axis=1)
    return chunk

chunksize = 80000  # Adjust chunk size as needed..
num_processes = cpu_count()  # Number of CPU cores


    # Concatenate the results into a single DataFrame or process them further

modified_chunks = []
for chunk in pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv', chunksize=80000):
    modified_chunk = chunk.apply(lambda row: evidence_dispersor(row,all_evidence_names), axis=1)
    print(type(modified_chunk))
    modified_chunks.append(modified_chunk)

# Concatenate the modified chunks into a single DataFrame
processed_data = pd.concat(modified_chunks, ignore_index=True)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [22]:
processed_data.to_csv(r'E:\MediNexus\Datasets\processed_data.csv')

In [33]:
processed_data

,,',",",-,.,/,0,1,2,3,...,z80.1,z82.49,z84.89,z92.25,â,è,é,ê,ô,û
0,0,0,0,NaN,0.0,NaN,0.0,0,0.0,0.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,NaN,NaN,NaN,0.0,0,0.0,NaN,...,0,0,0,0,NaN,0.0,0.0,0.0,NaN,NaN
2,0,0,0,NaN,0.0,NaN,0.0,0,0.0,NaN,...,0,0,0,0,NaN,NaN,0.0,NaN,0.0,NaN
3,0,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,NaN,NaN,NaN,NaN,0,0.0,NaN,...,0,0,0,0,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025597,0,0,0,NaN,0.0,NaN,0.0,0,0.0,NaN,...,0,0,0,0,NaN,0.0,0.0,NaN,0.0,NaN
1025598,0,0,0,NaN,0.0,NaN,0.0,0,0.0,NaN,...,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN
1025599,0,0,0,NaN,NaN,NaN,0.0,0,NaN,NaN,...,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN
1025600,0,0,0,NaN,NaN,NaN,0.0,0,NaN,NaN,...,0,0,0,0,NaN,NaN,0.0,NaN,0.0,NaN


In [ ]:
print(!)

In [ ]:
# Create a DataFrame with all 223 evidence columns
all_evidence_names = list(evidences_df['DiseaseName'].unique())
processed_data = pd.DataFrame(columns=['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'INITIAL_EVIDENCE'] + all_evidence_names)


def evidence_dispersor(row, all_evidence_names):
    for i in row['EVIDENCES']:
        filt1 = evidences_df['DiseaseName'] == i
        if filt1.any():
            if evidences_df.loc[filt1, 'datatype'].iloc[0] == 'B':
                row[i] = 1
            else:
                parts = i.split('_@_')
                if parts[1].isdigit():
                    print(parts[1])
                    row[parts[0]] = int(parts[1])

                else:
                    row[i] = 1
        else:
            row[i] = 0  # Set to 0 if evidence is not present

    # Set all other evidences to 0
    for ev_name in all_evidence_names:
        if ev_name not in row.index:
            row[ev_name] = 0

    return row

modified_chunks = []
for chunk in pd.read_csv(r'E:\MediNexus\Datasets\Symptoms2Dis\release_train_patients\release_train_patients.csv', chunksize=80000):
    modified_chunk = chunk.apply(lambda row: evidence_dispersor(row,all_evidence_names), axis=1)
    print(type(modified_chunk))
    modified_chunks.append(modified_chunk)

# Concatenate the modified chunks into a single DataFrame
processed_data = pd.concat(modified_chunks, ignore_index=True)